
Datasets : https://figshare.com/articles/dataset/Dataset_for_real-time_crack_detection_on_chicken_eggs/21568425

This data contain image (.jpg) and label (.xml) so we need to convert .xml to .txt for YOLOV8 training

This notebook is using ```ultralytics (yolov8n)``` for training 



In [ ]:
!pip install ultralytics

# Preparing data

Reference 
: https://towardsdatascience.com/convert-pascal-voc-xml-to-yolo-for-object-detection-f969811ccba5

In [ ]:
import xml.etree.ElementTree as ET
from glob import glob
import shutil
import os

In [ ]:
def xml_to_yolo_bbox(bbox, w, h):
    """
    Convert bounding box from xml format to YOLO format
    """
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]


def create_yolo_format(input_dir, output_dir):
    """
    create new directory for training YOLO model
    input_dir: directory of images and xml files
    output_dir: directory of new images and labels
    """
    img_output_dir = os.path.join(output_dir, "images")
    label_output_dir = os.path.join(output_dir, "labels")

    os.makedirs(img_output_dir, exist_ok=True) # create new directory
    os.makedirs(label_output_dir, exist_ok=True) 
 
    # get all jpg files
    all_images = glob(f"{input_dir}/*.jpg") # get all jpg files
    for image in all_images: 
        shutil.copy(image, img_output_dir) # copy image to new directory

    # get all xml files
    xml_files = glob(f"{input_dir}/*.xml")
    classes = []
    for file in xml_files:
        basename = os.path.basename(file) # get basename of xml file
        filename = os.path.splitext(basename)[0] # get filename without extension
        file_name = f"{input_dir}/{filename}.jpg" # get image file name
        if not os.path.exists(file_name): # check if image file exists
            print(f"{file_name} image does not exist!") 
            continue

        result = []
        tree = ET.parse(file) 
        root = tree.getroot() # get root of xml file
        width = int(root.find("size").find("width").text) # get width of image
        height = int(root.find("size").find("height").text) # get height of image

        for obj in root.findall("object"):
            label = obj.find("name").text # get label of object
            if label not in classes:
                classes.append(label) # add label to classes list
            index = classes.index(label) # get index of label
            pil_bbox = [int(x.text) for x in obj.find("bndbox")] # get bounding box of object
            yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height) # convert bounding box to YOLO format
            bbox_string = " ".join([str(x) for x in yolo_bbox]) # convert bounding box to string
            result.append(f"{index} {bbox_string}") # add label and bounding box to result list

        if result:
            with open(
                os.path.join(label_output_dir, f"{filename}.txt"), "w", encoding="utf-8" # create new txt file
            ) as f: 
                f.write("\n".join(result)) # write result to txt file

In [ ]:
create_yolo_format("train/train_100/", "train_yolo/")  # input_dir, output_dir
create_yolo_format("test/test_100/", "test_yolo/")  # input_dir, output_dir

In [ ]:
# Create yaml file which contains path to train, test, number of classes and class names 
import yaml

train_path = os.path.abspath("train_yolo/") # get absolute path of train_yolo directory
test_path = os.path.abspath("test_yolo/") 

data = dict(train=train_path, val=test_path, nc=2, names={0: "Crack", 1: "Egg"}) #this data contrain 2 classes: Crack and Egg

with open("egg_detection.yaml", "w") as f:
    yaml.dump(data, f)

## Training
```
As default we use yolov8n for training 
```
Reference : https://docs.ultralytics.com/


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from YAML
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolov8n.yaml").load("yolov8n.pt")  # build from YAML and transfer weights

In [ ]:
# Train the model
model.train(data="egg_detection.yaml", epochs=10, imgsz=1280)

## Validation

Define ```model``` for validation

In [ ]:
# Load a model
model = YOLO("runs/detect/train/weights/best.pt")  # load a custom model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

## Inference

In [ ]:
from PIL import Image, ImageDraw
from pathlib import Path

id2label = {0: "Crack", 1: "Egg"}

In [ ]:
def yolo_to_rectangle(bbox: tuple, img_width: int, img_height: int):
    """
    Converts a YOLO bounding box to a rectangle in pixel coordinates.
    """
    x_center, y_center, width, height = bbox
    x_pixel = x_center * img_width
    y_pixel = y_center * img_height
    x_top_left = x_pixel - (width * img_width) / 2
    y_top_left = y_pixel - (height * img_height) / 2
    x_bottom_right = x_pixel + (width * img_width) / 2
    y_bottom_right = y_pixel + (height * img_height) / 2
    return (x_top_left, y_top_left, x_bottom_right, y_bottom_right)


def predict_image(image_path: str, model: YOLO):
    """
    predict the image and return output in the following format
    boxes, keys, names, orig_img, orig_shape, path as ultralytics.yolo list
    """
    image_pic = Image.open(image_path)
    pred = model(image_pic)
    return pred


def visualize_prediction(predictions: list, image_path: str):
    """
    draw the prediction bounding box on the image
    predictions contains boxes, keys, names, orig_img, orig_shape, path
    return the image with blue bounding box and white text labels as PIL image
    """
    pred_img = Image.open(image_path)
    draw_prediction = ImageDraw.Draw(pred_img)
    boxes_predict = predictions[0].boxes
    boxes = boxes_predict.xyxy.tolist()
    scores = boxes_predict.conf.tolist()
    labels = boxes_predict.cls.tolist()
    for score, label, box in zip(scores, labels, boxes):
        x, y, x2, y2 = tuple(box)
        draw_prediction.rectangle((x, y, x2, y2), outline="blue", width=15)
        result = id2label[label]
        draw_prediction.text((x, y), result, fill="white", size=15)  # draw label text
    return pred_img


def visualize_pred_and_labels(pred_img: any, label_path: str):
    """
    draw the label bounding box on the prediction image
    return the image with red bounding box and yellow text labels as PIL image
    """
    with open(label_path, "r") as f:
        lines = f.readlines()
    id_label = []
    box_label = []
    for line in lines:
        data = line.strip().split()
        id_label.append(int(data[0]))
        box_label.append([float(x) for x in data[1:]])
    draw_label = ImageDraw.Draw(pred_img)
    for label, box in zip(id_label, box_label):
        x_top_left, y_top_left, x_bottom_right, y_bottom_right = yolo_to_rectangle(
            box, pred_img.width, pred_img.height
        )
        draw_label.rectangle(
            (x_top_left, y_top_left, x_bottom_right, y_bottom_right),
            outline="red",
            width=9,
        )
        result = id2label[label]
        draw_label.text(
            (x_top_left, y_top_left), result, fill="yellow", size=40
        )  # draw label text
    return pred_img

Define ```model``` and ```image_path``` for prediction

In [ ]:
model = YOLO("runs/detect/train/weights/best.pt")
image_path = "test_yolo/images/IMG_20220818_153638.jpg"  # path to image

In [ ]:
# predict image
predict_image = predict_image(image_path, model)

In [ ]:
# visualize prediction
predicted_image = visualize_prediction(predict_image, image_path)
predicted_image

In [ ]:
file_name = Path(image_path).stem
label_path = f"test_yolo/labels/{file_name}.txt"

# visualize prediction and label
predicted_image_and_label = visualize_pred_and_labels(predicted_image, label_path)
predicted_image_and_label